<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>

### Import necessary Libraries

In [154]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from pandas.io.json import json_normalize
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

### Foursquare Set-up

In [155]:
CLIENT_ID = 'CSWXP0BZFGWEJ1HTJAYRJ1IEW1T11E3PRR4BJBUDCGYZOH35' # your Foursquare ID
CLIENT_SECRET = 'BQ4HGWGQHUOCET3JUMMNTTJ5RVUJY5DW1EAQRYYLIV35CBY2' # your Foursquare Secret
VERSION = '20191016'
radius = 10000
ICLatitude = 41.6611
ICLongitude = -91.5302

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('Your Location:')
print ('LATITUDE - ', ICLatitude)
print ('LONGITUDE - ', ICLongitude)

Your credentails:
CLIENT_ID: CSWXP0BZFGWEJ1HTJAYRJ1IEW1T11E3PRR4BJBUDCGYZOH35
CLIENT_SECRET:BQ4HGWGQHUOCET3JUMMNTTJ5RVUJY5DW1EAQRYYLIV35CBY2
Your Location:
LATITUDE -  41.6611
LONGITUDE -  -91.5302


#### Define the corresponding URLs for CoffeeShops and Yoga Studio

In [156]:
CoffeeShopsCategoryId = '4bf58dd8d48988d1e0931735'
CoffeeShopUrl = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, ICLatitude, ICLongitude, VERSION, CoffeeShopsCategoryId, radius)
CoffeeShopUrl

'https://api.foursquare.com/v2/venues/search?client_id=CSWXP0BZFGWEJ1HTJAYRJ1IEW1T11E3PRR4BJBUDCGYZOH35&client_secret=BQ4HGWGQHUOCET3JUMMNTTJ5RVUJY5DW1EAQRYYLIV35CBY2&ll=41.6611,-91.5302&v=20191016&categoryId=4bf58dd8d48988d1e0931735&radius=10000'

In [157]:
GymCategoryId = '4bf58dd8d48988d102941735'
GymUrl = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, ICLatitude, ICLongitude, VERSION, GymCategoryId, radius)
GymUrl

'https://api.foursquare.com/v2/venues/search?client_id=CSWXP0BZFGWEJ1HTJAYRJ1IEW1T11E3PRR4BJBUDCGYZOH35&client_secret=BQ4HGWGQHUOCET3JUMMNTTJ5RVUJY5DW1EAQRYYLIV35CBY2&ll=41.6611,-91.5302&v=20191016&categoryId=4bf58dd8d48988d102941735&radius=10000'

### Create functions used to extract Foursquare results into a Pandas Dataframe

In [158]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

print ("get_category_type created...")

def getVenueDF(url):
    results = requests.get(url).json()
    venues = results['response']['venues']
    dataframe = json_normalize(venues)
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
    return dataframe_filtered

print ("get_venue_DF created...")

get_category_type created...
get_venue_DF created...


In [159]:
GymDF = getVenueDF(GymUrl)
print(GymDF.shape)
GymDF.head()
CoffeeShopDF = getVenueDF(CoffeeShopUrl)
print(CoffeeShopDF.shape)
CoffeeShopDF.head()

(8, 15)
(30, 15)


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Bruegger's Bagels,Bagel Shop,245 Iowa Ave,41.661014,-91.532027,"[{'label': 'display', 'lat': 41.66101403812128...",152,52240,US,Iowa City,IA,United States,"[245 Iowa Ave, Iowa City, IA 52240, United Sta...",NaN,525d64a511d2cdeeb7e5b1d8
1,The Java House,Coffee Shop,211 1/2 E Washington St,41.659944,-91.532536,"[{'label': 'display', 'lat': 41.65994360731062...",233,52240,US,Iowa City,IA,United States,[211 1/2 E Washington St (btw Dubuque St and L...,btw Dubuque St and Linn St,4b4cd4c4f964a520aec026e3
2,Caribou Coffee,Coffee Shop,245 Iowa Ave,41.661020,-91.532032,"[{'label': 'display', 'lat': 41.66102004968086...",152,52240,US,Iowa City,IA,United States,"[245 Iowa Ave, Iowa City, IA 52240, United Sta...",NaN,56fc566a498ef280f4025380
3,Starbucks,Coffee Shop,228 S. Clinton Street,41.658270,-91.534390,"[{'label': 'display', 'lat': 41.65827, 'lng': ...",469,52240,US,Iowa City,IA,United States,"[228 S. Clinton Street, Iowa City, IA 52240, U...",NaN,4b4f6639f964a520dc0427e3
4,High Ground Cafe,Coffee Shop,301 E Market St,41.663444,-91.531418,"[{'label': 'display', 'lat': 41.66344354179694...",279,52245,US,Iowa City,IA,United States,"[301 E Market St, Iowa City, IA 52245, United ...",NaN,51bce8de498ea91cb0c73c0b


### Map out Iowa City with Coffee Shops and Yoga Studios

In [160]:
map_IowaCity = folium.Map(location=[ICLatitude, ICLongitude], zoom_start=13)
for lat, lng, label in zip(CoffeeShopDF['lat'], CoffeeShopDF['lng'], CoffeeShopDF['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_IowaCity)  
for lat, lng, label in zip(GymDF['lat'], GymDF['lng'], GymDF['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ff6666',
        fill_opacity=0.7,
        parse_html=False).add_to(map_IowaCity)      
map_IowaCity

### Base on Proximity of Coffee Shops and Yoga Studios, I've identified three studios near Coffee Shops that can would be ideal for a pop-up shop.
#### These are mapped out below in green.

In [161]:
GymDF[(GymDF['name']=='Hothouse Yoga') | (GymDF['name']=='Zenergi Hot Yoga')]

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Hothouse Yoga,Yoga Studio,224 S Clinton St,Burlington St,41.658128,-91.534430,"[{'label': 'display', 'lat': 41.65812797900649...",482,52240,US,Iowa City,IA,United States,"[224 S Clinton St (Burlington St), Iowa City, ...",4bddab91587b2d7fd3975409
3,Hothouse Yoga,Yoga Studio,NaN,NaN,41.675907,-91.582533,"[{'label': 'display', 'lat': 41.67590734492047...",4653,NaN,US,Coralville,IA,United States,"[Coralville, IA, United States]",4cd7eb4d7bb06dcb84699fb2
4,Zenergi Hot Yoga,Gym,1705 S 1st Ave,NaN,41.642860,-91.504824,"[{'label': 'display', 'lat': 41.64286043188304...",2928,52240,US,Iowa City,IA,United States,"[1705 S 1st Ave, Iowa City, IA 52240, United S...",4b47d3e8f964a5208b4026e3


In [162]:
IdealLocations = GymDF[(GymDF['name']=='Hothouse Yoga') | (GymDF['name']=='Zenergi Hot Yoga')][['name','lat','lng']]
IdealLocations['name'] = 'Ideal Location'
IdealLocations.head()

,name,lat,lng
0,Ideal Location,41.658128,-91.534430
3,Ideal Location,41.675907,-91.582533
4,Ideal Location,41.642860,-91.504824


In [163]:
for lat, lng, label in zip(IdealLocations['lat'], IdealLocations['lng'], IdealLocations['name']):
    print(lat,lng,label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=25,
        popup=label,
        fill=True,
        fill_color='#ccff66',
        fill_opacity=0.3,
        parse_html=False).add_to(map_IowaCity)  
    
map_IowaCity

41.658127979006494 -91.53442985363114 Ideal Location
41.67590734492047 -91.58253322229731 Ideal Location
41.64286043188304 -91.50482430313113 Ideal Location
